In [6]:


file_path = 'MCMv331_NOAABB_V4_20200604_name_correct.m'
file_path = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Chem/GEOSChem/v13_latest/SubMechanisms/Sub_reduced_FUR.m'

In [15]:
def parse_reactions(file_path):
    reactions = []
    species_set = set()
    current_reaction = {}

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('Rnames'):
                if current_reaction:
                    reactions.append(current_reaction)
                current_reaction = {'Rnames': line.split('=')[1].strip(" ';"), 'Gstr': [], 'Products': {}}
            elif 'Gstr' in line:
                reactants = line.split('=')[1].strip(" {}';").replace("'", "").split()
                current_reaction['Gstr'].extend(reactants)
                species_set.update(reactants)
            elif line.startswith('f') and '=' in line:
                parts = line.split(';')
                for part in parts:
                    if '=' in part:
                        species = part.split('(')[0][1:]
                        change = part.split('=')[1].strip()
                        try:
                            if '+' in change:
                                amount = float(change.split('+')[1])
                            elif '-' in change:
                                amount = -float(change.split('-')[1])
                            else:
                                amount = float(change)
                        except ValueError:
                            continue  # Skip improperly formatted entries
                        
                        if species in current_reaction['Products']:
                            current_reaction['Products'][species] += amount
                        else:
                            current_reaction['Products'][species] = amount

    if current_reaction:  # Add the last reaction
        reactions.append(current_reaction)

    return reactions, species_set


def find_related_reactions(target_species, reactions):
    related_reactions = []
    related_species = set([target_species])
    to_process = [target_species]
    processed = set()

    while to_process:
        current_species = to_process.pop(0)
        if current_species in processed:
            continue
        for reaction in reactions:
            if current_species in reaction.get('Gstr', []) or current_species in reaction.get('Products', {}):
                if reaction not in related_reactions:
                    related_reactions.append(reaction)
                    related_species.update(reaction['Gstr'])
                    related_species.update(reaction['Products'].keys())
                    to_process.extend(reaction['Gstr'])
                    to_process.extend(reaction['Products'].keys())
        processed.add(current_species)

    return related_reactions, related_species

# File path to your MATLAB .m file
file_path = 'MCMv331_NOAABB_V4_20200604_name_correct.m'

# Parse the reactions from the file
reactions, all_species = parse_reactions(file_path)

# Print all reactions and species for debugging
#print("All species found:", all_species)
#print("All reactions found:")
#for reaction in reactions:
#    print(reaction)

# Specify the target species
target_species = 'FURAN'  # Change this to 'FURAN' or another if looking for a different target

# Find reactions related to the target species
related_reactions, related_species = find_related_reactions(target_species, reactions)
'''
print(f"Number of reactions related to {target_species}: {len(related_reactions)}")
for reaction in related_reactions:
    print(f"Reaction: {reaction['Rnames']}")
    print(f"  Reactants: {reaction.get('Gstr', [])}")
    print(f"  Products: {reaction.get('Products', {})}")
'''
print(f"Number of individual species related to {target_species}: {len(related_species)}")


Number of individual species related to FURAN: 15575
